In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path().cwd().parent))

In [3]:
from typing import Tuple

import pandas as pd

from plotting import plot_ts
from load_dataset import Dataset
from model import TimeSeriesPredictor

### Какие ряды будем тестировать?

* длинный ряд с сезонностью  
* короткий ряд с сезонностью  
* короткий ряд с сезонностью и трендом  
* случайное блуждание  
* средне зашумленный ряд
* "шумный" ряд  

In [4]:
ds = Dataset('data/dataset/')

In [5]:
long = ds['daily-min-temperatures.csv']
short_season = ds['hour_3019.csv'][300:]
short_season_trend = ds['international-airline-passengers.csv']
random_walk = ds['dow_jones_0.csv']
medium_noize = ds['hour_3426.csv'][300:]
full_noize = ds['day_1574.csv']

In [6]:
plot_ts(long)

Loading BokehJS ...

In [8]:
plot_ts(short_season)

Loading BokehJS ...

In [10]:
plot_ts(short_season_trend)

Loading BokehJS ...

In [12]:
plot_ts(random_walk)

Loading BokehJS ...

In [14]:
plot_ts(medium_noize)

Loading BokehJS ...

In [6]:
plot_ts(full_noize)

Loading BokehJS ...

### Какие модели будем тестировать?

* скользящее среднее
* экспоненциальное сглаживание
* autoArima
* линейная регрессия
* линейная регрессия с L1 регуляризацией (Ridge)
* RandomForeset
* градиентный бустинг


In [6]:
from estimators import RollingEstimator, ExponentialSmoothingEstimator
from pmdarima import auto_arima
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from statsmodels.tsa.arima_model import ARIMA

### По каким метрикам будем сравнивать?

* mse
* mae
* R2
* mape - если не будет ломаться на нулях
* mase

In [7]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score

from metrics import mean_absolute_percentage_error as mape
from metrics import mase

### По какой методике будем тестировать?

* 70% трейн, 30% тест
* Out-of-sample, чтобы посмотреть как модель предсказывает "вдолгую"
* In-Sample, чтобы посмотреть как модель предсказывает на одну точку вперед
* Для поиска гиперпараметров можно делать кроссвалидацию на тесте по метрике mse

### Задание 1. Напишите функцию, разбивающую на train и test

In [8]:
def train_test_split(ts: pd.Series, ratio: float = 0.7) -> Tuple[pd.Series]:
    split_indx = int(len(ts)*ratio)
    ts_train, ts_test = ts[:split_indx], ts[split_indx:]
    return ts_train, ts_test

### Зададим соответствие гранулярностей для наших рядов.

In [9]:
granularity_mapping = {
    'long': 'P1D',
    'short_season': 'PT1H',
    'short_season_trend': 'P1M',
    'random_walk': 'P1D',
    'medium_noize': 'PT1H',
    'full_noize': 'P1D'
}

### Задание 2. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через TimeSeriesPredictor.

* принмает на вход исходный ряд, гранулярность, количество лагов, модель, а также **kwargs, в которые мы будем передавать параметры модели

* разбивает ряд на train/test

* создает инстанс TimeSeriesPredictor с нужными параметрами

* обучает предиктор на трейне

* делает out_of_sample и in_sample прогноз

* возвращает train, test, in_sample, out_of_sample

In [10]:
def check_params_model(params_model, params):
    found={}
    for key, value in params.items():
        for param_key in params_model.keys():
            if (param_key.find(key) != -1):
                found.update({key:value})
    return found

In [11]:
def make_pipeline(ts: pd.Series, granularity: str, num_lags: int, model: callable, **kwargs) -> Tuple[pd.Series]:
    train, test = train_test_split (ts)
    predictor = TimeSeriesPredictor(
                                    num_lags=num_lags,
                                    granularity=granularity,
                                    model=model)

    if len(train)<num_lags: #проверяем длину лага на длину тестовой выборки
        return train, test,0,0
    params = check_params_model(predictor.get_params(), kwargs) #отбрасываем параметры не поддерживаемые моделью
    predictor.set_params(**params)
    predictor.fit(train)
    out_of_sample = predictor.predict_next(train, n_steps=len(test))
    in_sample = predictor.predict_batch(train, test)                         

    return train, test, in_sample, out_of_sample

### Задание 3. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через auto_arima

* функция должна принимать исходный временной ряд, период сезонности, параметры дифференцирования d, D и boolean параметр seasonal, данные параметры будут являться для нас гиперпараметрами, все остальное за нас должна найти auto_arima

* разбивает на train, test

* обучает arima на train при помощи вызова функции auto_arima из библиотеки pmdarima с переданными параметрами и со следующими зафиксированными параметрами: `max_p=3, max_q=3, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True`

* в качестве out_of_sample прогноза просто вызовите метод predict

* в качестве in_sample прогноза обучите модель заново на всём ряде методом `fit`, вызовите метод predict_in_sample и в качестве прогноза возьмите `in_sample_predictions(-len(test):)`

* возвращает train, test, in_sample, out_of_sample (не забудьте сделать их pd.Series с нужным индексом!!)

In [12]:
from pmdarima import auto_arima
def make_pipeline_arima(ts: pd.Series, num_lags: int, d: int = 1, D: int = 1, seasonal: bool = True) -> Tuple[pd.Series]:
    train, test = train_test_split (ts)
    model = auto_arima(
                train, start_p=0, start_q=0,
                max_p=3, max_q=3, start_d=0, start_D=0,
                max_d=2, max_D=2,
                m=num_lags, start_P=0, start_Q=0,
                seasonal=seasonal, trace=True,
                error_action='ignore',
                suppress_warnings=True,
                stepwise=True)
    out_of_sample = model.predict(len(test))
    
    model.fit(ts)
    in_sample = model.predict_in_sample()[-len(test):]

    in_sample = pd.Series(in_sample, test.index)
    out_of_sample = pd.Series(out_of_sample, test.index)
    order = model.order
    del model
    return train, test, in_sample, out_of_sample, order

### Задание 4. "Прогоните" все алгоритмы на всех рядах и получите сводную таблицу результатов по всем метрикам, постройте также графики прогнозов. 

In [13]:
from itertools import product
from plotting import plot_ts
import numpy as np


granularity_mapping = {
    'long': 'P1D',
    'short_season': 'PT1H',
    'short_season_trend': 'P1M',
    'random_walk': 'P1D',
    'medium_noize': 'PT1H',
    'full_noize': 'P1D'}

ts_dict = {
    'long': long,
    'short_season': short_season,
    'short_season_trend': short_season_trend,
    'random_walk': random_walk,
    'medium_noize': medium_noize,
    'full_noize': full_noize}

# ts_dict = {
#     'long': long}

models = [LinearRegression, Ridge, RandomForestRegressor, GradientBoostingRegressor]
param_grid = {
    'max_depth': [6, 12],
    'n_estimators': [50, 500, 1000],
    'num_lags': [7,12,24,60,30,180,365],
    'normalize': [True, False],
    'seasonal': [0]}

param_grid_arima = {
    'max_depth': [0],
    'n_estimators': [0],
    'num_lags': [0],#[30,180,365], #[7,12,24,60,30,180,365],
    'normalize': [0],
    'seasonal': [True, False]}    

    

In [14]:
from typing import Tuple, Dict

def hyperparameters_search(ts, ts_name, param_grid, model='', arima=False, verbose=False):
    best_mse_in = 0
    best_mse_out = 0

    for max_depth, n_estimators, num_lags, normalize, seasonal in product(*param_grid.values()):
        num_lags = extract_season_lag(ts, granularity_mapping[ts_name])

        if arima:
            try:
                train, test, in_sample, out_of_sample, arima_order = make_pipeline_arima(  ts=ts,
                                                                    num_lags=num_lags,
                                                                    seasonal=seasonal)
            except Exception:
                in_sample = 0
        
        else:
            train, test, in_sample, out_of_sample = make_pipeline(  ts=ts,
                                                                granularity=granularity_mapping[ts_name],
                                                                num_lags=num_lags,
                                                                model=model,
                                                                model__max_depth=max_depth,
                                                                model__n_estimators=n_estimators,
                                                                model__normalize=normalize)

        if type(in_sample) == int: continue #Ошибка в лаге, пропускаем расчет

        mse_in = MSE(test, in_sample)
        mse_out = MSE(test, out_of_sample)
        mae_in = MAE(test, in_sample)
        mae_out = MAE(test, out_of_sample)
        r2_score_in = r2_score(test, in_sample)
        r2_score_out = r2_score(test, out_of_sample)
        mape_in = mape(test, in_sample) 
        mape_out = mape(test, out_of_sample)
        mase_in = mase(in_sample, test)
        mase_out = mase(out_of_sample, test)


        if mse_in < best_mse_in or not best_mse_in:
           best_mse_in = mse_in
           best_param_in = {
                            'type': 'in_sample',
                            'ts_name': ts_name,                    
                            'model': 'Arima'+str(arima_order) if arima else model.__name__,
                            'mse': mse_in,
                            'mae': mae_in,
                            'r2_score': r2_score_in,
                            'mape': mape_in,
                            'mase': mase_in,
                            'max_depth': max_depth,
                            'n_estimators': n_estimators,
                            'normalize': normalize,
                            'seasonal': seasonal,
                            'num_lags': num_lags,
                            'preds': in_sample}

        if mse_out < best_mse_out or not best_mse_out:
           best_mse_out = mse_out
           best_param_out = {
                            'type': 'out_of_sample',
                            'ts_name': ts_name,                                                
                            'model': 'Arima'+str(arima_order) if arima else model.__name__,
                            'mse': mse_out,
                            'mae': mae_out,
                            'r2_score': r2_score_out,
                            'mape': mape_out,
                            'mase': mase_out,
                            'max_depth': max_depth,
                            'n_estimators': n_estimators,
                            'normalize': normalize,
                            'seasonal': seasonal,                                                      
                            'num_lags': num_lags,
                            'preds': out_of_sample}

    return best_param_in, best_param_out, train, test

In [15]:
# извлекаем лаг сезонности
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf

def extract_season_lag(data: pd.Series, granularity) -> int:
    season=granularity[-1:]

    data_per_day = pd.Series({'value':np.NaN}, index=pd.date_range(
                                start=data[:1].index.values[0],
                                end=data[-1:].index.values[0],
                                freq=season+'S' if season=='M' else season))
    data = data.groupby(data.index).first() #убираем дубликаты
    data = data_per_day.combine_first(data) #соединяем 
    data.index = pd.DatetimeIndex(data.index).to_period(season)
    data.fillna(method='pad', inplace=True) #заполняем NaN предыдущими значениями
    data.index=data.index.to_timestamp()

    result = seasonal_decompose(data, model = 'additive')

    data = np.diff((acf(result.seasonal)))
    extrem = []
    period = []
    extrem_flag=True
    for n in range(len(data)): #ищем экстремум функции
        if data[n]<0 and extrem_flag:
            extrem.append(data[n])
            extrem_flag=False
        if data[n]>0 and not extrem_flag: extrem_flag=True

    for n in extrem:
        period = np.append(period, np.where(data == n)[0][0])
        
    return int(period[1]) #берем первый лаг (нулевой пропускаем)

In [225]:
#liner regretion
df_best_model = pd.DataFrame(columns=['ts_name', 'model', 'type', 'mse', 'mae', 'r2_score', 'mape', 'mase', 'max_depth', 'n_estimators',                             'normalize', 'num_lags', 'preds'])
for ts_name, ts in ts_dict.items():
    for model in models:
        best_param_in, best_param_out, train, test = hyperparameters_search(ts, ts_name, model, param_grid, arima=False)
        df_best_model = df_best_model.append([best_param_in, best_param_out], ignore_index=True)        

In [233]:
#ARIMA
for ts_name, ts in ts_dict.items():
    best_param_in, best_param_out, train, test = hyperparameters_search(ts, ts_name, param_grid_arima, arima=True)
    df_best_model = df_best_model.append([best_param_in, best_param_out], ignore_index=True)   

l_order=(2, 0, 2, 7); AIC=12270.828, BIC=12323.436, Fit time=19.855 seconds
Fit ARIMA: order=(3, 0, 1) seasonal_order=(2, 0, 2, 7); AIC=11893.198, BIC=11951.652, Fit time=40.204 seconds
Fit ARIMA: order=(3, 0, 1) seasonal_order=(1, 0, 2, 7); AIC=11862.920, BIC=11915.529, Fit time=27.240 seconds
Fit ARIMA: order=(3, 0, 1) seasonal_order=(1, 0, 1, 7); AIC=11856.529, BIC=11903.293, Fit time=20.017 seconds
Fit ARIMA: order=(3, 0, 1) seasonal_order=(0, 0, 0, 7); AIC=11853.271, BIC=11888.344, Fit time=8.606 seconds
Fit ARIMA: order=(2, 0, 1) seasonal_order=(0, 0, 0, 7); AIC=11890.647, BIC=11919.874, Fit time=6.796 seconds
Fit ARIMA: order=(3, 0, 0) seasonal_order=(0, 0, 0, 7); AIC=12080.897, BIC=12110.124, Fit time=1.289 seconds
Fit ARIMA: order=(3, 0, 2) seasonal_order=(0, 0, 0, 7); AIC=11855.393, BIC=11896.311, Fit time=9.591 seconds
Fit ARIMA: order=(2, 0, 0) seasonal_order=(0, 0, 0, 7); AIC=12173.453, BIC=12196.835, Fit time=1.377 seconds
Fit ARIMA: order=(3, 0, 1) seasonal_order=(1, 0, 

In [242]:
df_best_model.head()

,ts_name,model,type,mse,mae,r2_score,mape,mase,max_depth,n_estimators,normalize,num_lags,preds,seasonal
0,long,LinearRegression,in_sample,5.222552,1.791274,0.672680,19.985880,0.889396,6,50,True,24,1981-01-01 1988-01-01 15.472612 1988-01-02 ...,NaN
1,long,LinearRegression,out_of_sample,11.943355,2.733802,0.251458,29.285506,1.357377,6,50,True,365,1988-01-01 12.547939 1988-01-02 13.23444...,NaN
2,long,Ridge,in_sample,5.222550,1.791271,0.672680,19.985941,0.889395,6,50,False,24,1981-01-01 1988-01-01 15.472451 1988-01-02 ...,NaN
3,long,Ridge,out_of_sample,8.052230,2.200943,0.495331,24.036458,1.092804,6,50,True,365,1988-01-01 13.050864 1988-01-02 13.12033...,NaN
4,long,RandomForestRegressor,in_sample,4.951559,1.751149,0.689664,18.784094,0.869474,6,50,False,180,1981-01-01 1988-01-01 15.054796 1988-01-02 ...,NaN


In [235]:
# df_best_model.to_pickle('df_best_model_with_Arima.pkl')

In [16]:
df_best_model = pd.read_pickle('df_best_model_with_Arima.pkl')

In [17]:
#Выбираем лучшие модели
df_top_best_model = pd.DataFrame(columns=['ts_name', 'model', 'type', 'mse', 'mae', 'r2_score', 'mape', 'mase', 'max_depth', 'n_estimators', 'num_lags', 'preds'])
for ts_name in ts_dict.keys():
    for type_sample in ['in_sample', 'out_of_sample']:
        mask = (df_best_model['ts_name'].values == ts_name) & (df_best_model['type'].values == type_sample)
        df_top_best_model = df_top_best_model.append(df_best_model[mask].sort_values(['mse', 'mae', 'r2_score', 'mape', 'mase']).head(1), ignore_index=True)

In [18]:
df_top_best_model.head()

,ts_name,model,type,mse,mae,r2_score,mape,mase,max_depth,n_estimators,num_lags,preds,normalize,seasonal
0,long,RandomForestRegressor,in_sample,4.951559,1.751149,0.689664,18.784094,0.869474,6,50,180,1981-01-01 1988-01-01 15.054796 1988-01-02 ...,False,NaN
1,long,Ridge,out_of_sample,8.052230,2.200943,0.495331,24.036458,1.092804,6,50,365,1988-01-01 13.050864 1988-01-02 13.12033...,True,NaN
2,short_season,"Arima(3, 0, 2)",in_sample,481.682115,13.737605,0.848239,53.540198,0.634700,0,0,24,2019-05-30T10:00:00 2019-06-26 20:00:00 38...,0,False
3,short_season,"Arima(3, 0, 2)",out_of_sample,1039.658455,24.183593,0.672441,207.038837,1.117322,0,0,24,2019-05-30T10:00:00 2019-06-26 20:00:00 38...,0,False
4,short_season_trend,LinearRegression,in_sample,216.990089,11.335684,0.963072,2.698409,0.276577,6,50,30,1949-01 1957-06-01 424.556242 1957-07-01 ...,False,NaN


In [258]:
#рисуем графики и т.д.
for n in range(len(df_top_best_model)):
    train, test = train_test_split(ts_dict[df_top_best_model.iloc[n]['ts_name']])
    print (df_top_best_model.iloc[n][:11])
    plot_ts(test, df_top_best_model.iloc[n]['preds'], legends=['test', 'preds'])


ts_name                          long
model           RandomForestRegressor
type                        in_sample
mse                          4.951559
mae                          1.751149
r2_score                     0.689664
mape                        18.784094
mase                         0.869474
max_depth                           6
n_estimators                       50
num_lags                          180
Name: 0, dtype: object


Loading BokehJS ...

ts_name                  long
model                   Ridge
type            out_of_sample
mse                   8.05223
mae                  2.200943
r2_score             0.495331
mape                24.036458
mase                 1.092804
max_depth                   6
n_estimators               50
num_lags                  365
Name: 1, dtype: object


Loading BokehJS ...

ts_name           short_season
model           Arima(3, 0, 2)
type                 in_sample
mse                 481.682115
mae                  13.737605
r2_score              0.848239
mape                 53.540198
mase                    0.6347
max_depth                    0
n_estimators                 0
num_lags                    24
Name: 2, dtype: object


...

Loading BokehJS ...

ts_name             full_noize
model           Arima(0, 0, 0)
type                 in_sample
mse                   0.000032
mae                   0.004697
r2_score              -0.01062
mape                 12.905194
mase                  0.706696
max_depth                    0
n_estimators                 0
num_lags                     2
Name: 10, dtype: object


Loading BokehJS ...

ts_name                        full_noize
model           GradientBoostingRegressor
type                        out_of_sample
mse                              0.000031
mae                              0.004658
r2_score                          0.02524
mape                            12.937199
mase                             0.700839
max_depth                               6
n_estimators                          500
num_lags                               12
Name: 11, dtype: object


Loading BokehJS ...